In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../src/')

In [2]:
from package.logger import Timed, rlog, setup
from package import storage
setup("INFO")

In [3]:
import os

import folium
import geopandas as gpd
import pandas as pd

from package import strtime
from package.osm import osm
from package.mcr.data import NetworkType, OSMData
from package.mcr5.h3 import (
    get_h3_cells_for_nodes,
    get_h3_cells_for_bbox,
    plot_h3_cells_on_folium,
)
from package.mcr5.h3_osm_interaction import get_location_mappings_for_cells
from package.mcr5.mcr5 import MCR5
from package.osm.osm import add_nearest_osm_node_id
from package.mcr5.labels import read_labels_for_nodes
from package.minute_city import minute_city

from package.geometa import GeoMeta
from package.mcr.data import NetworkType

In [4]:
geo_meta_path = "../data/geometa.pkl"
city_id = "Koeln"
stops = "../data/cleaned/stops.csv"
structs = "../data/structs.pkl"
bicycle_location_path = "../data/bicycle_locations/2022-11-01_09_01_00.csv"

In [5]:
geo_meta = GeoMeta.load(geo_meta_path)
geo_data = OSMData(
	geo_meta,
	city_id,
	additional_network_types=[NetworkType.DRIVING],
)
nodes = geo_data.osm_nodes
edges = geo_data.osm_edges
driving_nodes, _, _ = geo_data.additional_networks[NetworkType.DRIVING]

[17:55:23] INFO     Using existing OSM data                                                              ]8;id=881227;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=160492;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#106\106]8;;\

           INFO     Loading OSM network from cache (walking)                                              ]8;id=699904;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=794852;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#69\69]8;;\

[17:56:03] INFO     Using existing OSM data                                                              ]8;id=948934;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=651494;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#106\106]8;;\

           INFO     Loading OSM network from cache (driving)                                              ]8;id=865520;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=870211;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#69\69]8;;\

In [6]:
h3_cells = get_h3_cells_for_nodes(nodes[["lat", "lon"]].to_dict("records"), 9)
driving_h3_cells = get_h3_cells_for_nodes(driving_nodes[["lat", "lon"]].to_dict("records"), 9)
h3_cells = h3_cells.intersection(driving_h3_cells)

In [7]:
location_mappings, invalid_h3_cells = get_location_mappings_for_cells(list(h3_cells), nodes, max_tries=20)
invalid_h3_cells

[]

In [8]:
car_location_mappings, invalid_h3_cells = get_location_mappings_for_cells(
    list(h3_cells), driving_nodes, max_tries=40
)

In [9]:
from command.step_config import (
    get_bicycle_public_transport_config,
    get_car_only_config,
    get_bicycle_only_config,
	get_public_transport_only_config,
    get_walking_only_config,
)


In [10]:
configs = {}
initial_steps, repeating_steps = get_bicycle_public_transport_config(
    geo_meta_path=geo_meta_path,
    city_id=city_id,
    bicycle_price_function="next_bike_no_tariff",
    bicycle_location_path=bicycle_location_path,
    structs_path=structs,
    stops_path=stops,
)
configs["bicycle_public_transport"] = {
    "init_kwargs": {
        "initial_steps": initial_steps,
        "repeating_steps": repeating_steps,
    },
    "location_mappings": location_mappings,
    "max_transfers": 2,
}

initial_steps, repeating_steps = get_car_only_config(
    geo_meta_path,
    city_id,
)
configs["car"] = {
    "init_kwargs": {
        "initial_steps": initial_steps,
        "repeating_steps": repeating_steps,
    },
    "location_mappings": car_location_mappings,
    "max_transfers": 1,
}


initial_steps, repeating_steps = get_bicycle_only_config(
    geo_meta_path=geo_meta_path,
    city_id=city_id,
    bicycle_price_function="next_bike_no_tariff",
    bicycle_location_path=bicycle_location_path,
)
configs["bicycle"] = {
    "init_kwargs": {
        "initial_steps": initial_steps,
        "repeating_steps": repeating_steps,
    },
    "location_mappings": location_mappings,
    "max_transfers": 2,
}


initial_steps, repeating_steps = get_public_transport_only_config(
    geo_meta_path=geo_meta_path,
    city_id=city_id,
    structs_path=structs,
    stops_path=stops,
)
configs["public_transport"] = {
    "init_kwargs": {
        "initial_steps": initial_steps,
        "repeating_steps": repeating_steps,
    },
    "location_mappings": location_mappings,
    "max_transfers": 2,
}

initial_steps, repeating_steps = get_walking_only_config(
    geo_meta_path=geo_meta_path,
    city_id=city_id,
)
configs["walking"] = {
    "init_kwargs": {
        "initial_steps": initial_steps,
        "repeating_steps": repeating_steps,
    },
    "location_mappings": location_mappings,
    "max_transfers": 0,
}


[17:56:28] INFO     Using existing OSM data                                                              ]8;id=262077;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=523;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#106\106]8;;\

           INFO     Loading OSM network from cache (walking)                                              ]8;id=551803;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=897922;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#69\69]8;;\

[17:57:09] INFO     Using existing OSM data                                                              ]8;id=797016;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=222237;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#106\106]8;;\

           INFO     Reading OSM network (cycling)                                                         ]8;id=197038;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=576917;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#74\74]8;;\

[17:58:07] INFO     Reading OSM network (cycling) done (58.98 seconds)                                    ]8;id=663277;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=718612;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#74\74]8;;\

...

           INFO     Reading stops done (0.09 seconds)                                       ]8;id=501216;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/steps/public_transport.py\public_transport.py]8;;\:]8;id=696293;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr/steps/public_transport.py#144\144]8;;\

[18:09:42] INFO     Using existing OSM data                                                              ]8;id=848169;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=937199;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#106\106]8;;\

           INFO     Loading OSM network from cache (walking)                                              ]8;id=48612;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=358429;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#69\69]8;;\

[18:10:22] INFO     Fetching POI for runtime optimization                                        ]8;id=41905;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/command/step_config.py\step_config.py]8;;\:]8;id=620662;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/command/step_config.py#165\165]8;;\

           INFO     Fetching POI for runtime optimization done (0.03 seconds)                    ]8;id=553316;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/command/step_config.py\step_config.py]8;;\:]8;id=269539;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/command/step_config.py#165\165]8;;\

In [11]:
for key, config in configs.items():
    mcr5 = MCR5(**config["init_kwargs"])

    rlog.info(f"Running MCR5 for {key}")
    print(f"Running MCR5 for {key}")

    mcr5_output_path = os.path.join("../data/mcr5/Koeln", key)

    location_mappings = config["location_mappings"]

    errors = mcr5.run(
        location_mappings, start_time="08:00:00", output_dir=mcr5_output_path, max_transfers=config["max_transfers"]
    )


[18:10:36] INFO     Running MCR5 for bicycle_public_transport                                        ]8;id=869359;file:///tmp/ipykernel_76778/494235893.py\494235893.py]8;;\:]8;id=635643;file:///tmp/ipykernel_76778/494235893.py#4\4]8;;\

Running MCR5 for bicycle_public_transport


Starting:   0%|          | 0/3590 [00:00<?, ?it/s]

[18:26:19] INFO     All processes finished.                                                             ]8;id=261090;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr5/mcr5.py\mcr5.py]8;;\:]8;id=178169;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr5/mcr5.py#100\100]8;;\

           INFO     Running MCR5 for car                                                             ]8;id=509335;file:///tmp/ipykernel_76778/494235893.py\494235893.py]8;;\:]8;id=702994;file:///tmp/ipykernel_76778/494235893.py#4\4]8;;\

...

[19:03:51] INFO     All processes finished.                                                             ]8;id=435364;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr5/mcr5.py\mcr5.py]8;;\:]8;id=117986;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr5/mcr5.py#100\100]8;;\

           INFO     Running MCR5 for walking                                                         ]8;id=82506;file:///tmp/ipykernel_76778/494235893.py\494235893.py]8;;\:]8;id=311917;file:///tmp/ipykernel_76778/494235893.py#4\4]8;;\

Running MCR5 for walking


Starting:   0%|          | 0/3590 [00:00<?, ?it/s]

[19:14:11] INFO     All processes finished.                                                             ]8;id=453419;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr5/mcr5.py\mcr5.py]8;;\:]8;id=184593;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/mcr5/mcr5.py#100\100]8;;\

In [12]:
import pickle
errors = storage.read_any_dict("../data/mcr5/Koeln/bicycle_public_transport/errors.pkl")
print(errors[0]["logs"])

IndexError: list index out of range

**small area**  
2s with limits  
20s without limits


**full area**  
2s with limits  
9m58s without limits